In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [3]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [4]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [5]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
                #print('\n',annotation_lines[i],input_shape)                
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

In [6]:
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

Actual code

In [7]:
ss=800
annotation_path = 'train550.txt'
log_dir = 'logs/005/'
data_path='data/players2'
anchors_path='data/players2/yolo_anchors.txt'
classes_path='data/players2/player_classes.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
#YOLO_ANCHORS = np.array(((0.091667, 0.03125), (0.077778, 0.019531), (0.033228 , 0.027344)))
YOLO_ANCHORS = np.array(
    ((0.16389 , 0.038281), (0.15278, 0.036719), (0.10278 , 0.039063),
     (0.12917 , 0.027344), (0.094444, 0.029687),(0.091667  ,0.020313),
    ( 0.083333, 0.017969),(0.055556 , 0.014063),(0.034333, 0.01875)))
anchors = YOLO_ANCHORS*ss
anchors_number = YOLO_ANCHORS.shape[0]
anchors=anchors[[8,7,6,5,4,3,2,1,0],:]
#print(anchors)

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12162945054844188003
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4962752921
locality {
  bus_id: 1
  links {
  }
}
incarnation: 64881824264122743
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [13]:
from keras.utils import multi_gpu_model
input_shape = (ss,ss)
model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/yolo_weights.h5')
#model = multi_gpu_model(model, gpus=1)

Create YOLOv3 model with 9 anchors and 1 classes.


c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\keras\engine\topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 18) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\keras\engine\topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\keras\engine\topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 18) vs (255, 512, 1, 1)).
  weight_values[i].shape))
c:\users\popeye\appdata\local\programs\python\python35\lib\site-packages\keras\engine\topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
c:\users\popey

Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.


ValueError: For multi-gpu usage to be effective, call `multi_gpu_model` with `gpus >= 2`. Received: `gpus=1`

In [12]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [10]:
logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [11]:
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(42)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

In [12]:
model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

In [13]:
batch_size = 12
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=50,
        initial_epoch=0,
        callbacks=[logging, checkpoint])
model.save_weights(log_dir + 'trained_weights_stage_1.h5')

Train on 495 samples, val on 55 samples, with batch size 12.
Epoch 1/50
41/41 [==============================] - 171s 4s/step - loss: 9331.2992 - val_loss: 1355.5556
Epoch 2/50
41/41 [==============================] - 152s 4s/step - loss: 819.7574 - val_loss: 646.6353
Epoch 3/50
41/41 [==============================] - 154s 4s/step - loss: 502.9515 - val_loss: 476.2244
Epoch 4/50
41/41 [==============================] - 138s 3s/step - loss: 367.5665 - val_loss: 359.5885
Epoch 5/50
41/41 [==============================] - 151s 4s/step - loss: 301.9016 - val_loss: 292.0195
Epoch 6/50
41/41 [==============================] - 152s 4s/step - loss: 259.0209 - val_loss: 259.9199
Epoch 7/50
41/41 [==============================] - 151s 4s/step - loss: 229.1506 - val_loss: 228.0670
Epoch 8/50
41/41 [==============================] - 150s 4s/step - loss: 203.7086 - val_loss: 204.4378
Epoch 9/50
41/41 [==============================] - 152s 4s/step - loss: 188.0695 - val_loss: 181.3299
Epoch 10/5

In [14]:
model.save_weights(log_dir + 'trained_weights_stage_1.h5')

In [15]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.

for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')

batch_size = 1 # note that more GPU memory is required after unfreezing the body
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=100,
    initial_epoch=50,
    callbacks=[logging, checkpoint, reduce_lr, early_stopping])
model.save_weights(log_dir + 'trained_weights_final.h5')

# Further training if needed.

Unfreeze all of the layers.
Train on 495 samples, val on 55 samples, with batch size 1.
Epoch 51/100
495/495 [==============================] - 318s 643ms/step - loss: 65.4293 - val_loss: 61.0577
Epoch 52/100
495/495 [==============================] - 301s 608ms/step - loss: 60.0222 - val_loss: 103.8720
Epoch 53/100
495/495 [==============================] - 301s 609ms/step - loss: 58.8300 - val_loss: 57.0399
Epoch 54/100
495/495 [==============================] - 301s 608ms/step - loss: 58.9797 - val_loss: 57.3891
Epoch 55/100
495/495 [==============================] - 301s 609ms/step - loss: 57.3478 - val_loss: 59.2997
Epoch 56/100
495/495 [==============================] - 302s 610ms/step - loss: 56.9181 - val_loss: 60.0521

Epoch 00056: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 57/100
495/495 [==============================] - 302s 610ms/step - loss: 54.2230 - val_loss: 56.9067
Epoch 58/100
495/495 [==============================] - 302s 611ms/step - 